### Dependencies

In [1]:
!git clone "https://github.com/suyash/ContextualDecomposition.git" && mv ContextualDecomposition/cd ./cd

Cloning into 'ContextualDecomposition'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 90 (delta 45), reused 87 (delta 42), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [2]:
!curl -L -o "job_dir.zip" "https://drive.google.com/uc?export=download&id=1wkygiEOc2T9LFbD4fcO__9tXGVkxu0qu" && unzip -q -d "job_dir" "job_dir.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    204      0 --:--:--  0:00:01 --:--:--   204
100 11.0M    0 11.0M    0     0  4318k      0 --:--:--  0:00:02 --:--:-- 4318k


### Imports

In [3]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf

from cd.acd import conv1d_acd_decomposition, agglomerate_acd_1d_decomposition
from cd.preprocess import create_table, create_inv_table

In [5]:
model = tf.keras.models.load_model("job_dir/saved_model/best")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     915904      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 256)    33024       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    24704       embedding[0][0]                  
______________________________________________________________________________________________

In [0]:
tokens = np.load("job_dir/tokens.npy")
table = create_table(tokens)
inv_table = create_inv_table(tokens)

In [0]:
s = "the longer the movie goes , the worse it gets , but it 's actually pretty good in the first few minutes ."

In [0]:
text = s.split()

In [9]:
inp = table.lookup(tf.constant(s.split()))
inp = tf.expand_dims(inp, 0)
inp

<tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[   1, 1752,    1,   19,  336,    2,    1,  552,   12,  283,    2,
          22,   12,    8,  301,  313,   46,   11,    1,  167,  169,  164,
           6]], dtype=int32)>

In [10]:
comps, comp_scores = conv1d_acd_decomposition(inp, model)

/content/cd/acd.py:29: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 99.5)
/content/cd/acd.py:15: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 59)
/content/cd/acd.py:21: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 59)
/content/cd/acd.py:24: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 49)


In [0]:
tree = agglomerate_acd_1d_decomposition(comps, comp_scores)

In [12]:
for i, l in enumerate(tree):
    print("Level", i)

    for s in l:
        print(" ".join(text[s[0][0]:s[0][1]]), ":", s[1], ", children:", s[2])

    print("--------------------------------------------------------")

Level 0
the : 0.023723755031824112 , children: []
longer : 0.024546079337596893 , children: []
the : 0.021861907094717026 , children: []
movie : 0.033330559730529785 , children: []
goes : 0.03823969513177872 , children: []
, : 0.2517704367637634 , children: []
the : 0.5991467237472534 , children: []
worse : -13.187629699707031 , children: []
it : -1.280884027481079 , children: []
gets : -1.7463042736053467 , children: []
, : 1.6393227577209473 , children: []
but : 2.7335333824157715 , children: []
it : 1.299285650253296 , children: []
's : 0.7545077204704285 , children: []
actually : 1.4517323970794678 , children: []
pretty : 1.8054380416870117 , children: []
good : 2.471308946609497 , children: []
in : -0.6136900782585144 , children: []
the : -0.2862776219844818 , children: []
first : 0.9100427627563477 , children: []
few : 0.1771470606327057 , children: []
minutes : -0.024332180619239807 , children: []
. : 0.04344141483306885 , children: []
-------------------------------------------